# Predicciones Privadas con Syft Keras

## Paso 1: Aprendizaje Público

Bienvenidos a este tutorial! En los siguientes notebooks aprenderán a cómo proveer predicciones privadas. Por predicciones privadas, nos referimos a que los datos están constantemente encriptados durante todo el proceso. En ningún punto el usuario comparte datos originales, sino mas bien encriptados (es decir, compartidos en secreto). Para proveer estas predicciones privadas, Syft Keras utiliza una librería llamada [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted). TF Encrypted combina estados del arte en técnicas de criptografía y aprendizaje de máquinas, pero no se tienen que preocupar por esto y en su lugar pueden enfocarse en la aplicación de aprendizaje de máquinas.

Podemos empezar a proveer predicciones privadas con solo 3 pasos:
- **Paso 1**: Entrena tu modelo con Keras normal.
- **Paso 2**: Asegura y cuelga tu modelo de aprendizaje de máquinas (servidor).                                               
- **Paso 3**: Consulta al modelo seguro para recibir la predicción privada (cliente).

Bueno, vamos a seguir estos 3 pasos para que puedan desplegar servicios de aprendizaje de máquinas impactantes sin sacrificar la privacidad del usuario o la seguridad del modelo.

Autores:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

Traductores:
- Daniel Firebanks-Quevedo - GitHub: [@thefirebanks](https://www.github.com/thefirebanks)

## Entrena tu modelo con Keras

Para utilizar técnicas de aprendizaje de máquinas que preservan la privacidad en tus proyectos, no deberías tener que aprender un nuevo framework. Si tienes conocimientos básicos de [Keras](https://keras.io/), puedes empezar usando estas técnicas con Syft Keras. Si nunca antes has utilizado Keras, puedes aprender un poco más en la [documentación](https://keras.io). 

Antes de proveer predicciones privadas, el primer paso es entrenar tu modelo con la versión normal de Keras. Como un ejemplo, vamos a entrenar un modelo para clasificar dígitos escritos a mano. Para entrenar este modelo usaremos el conjunto de datos canónico de [MNIST](http://yann.lecun.com/exdb/mnist/).

Usaremos [este ejemplo](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) tomado del repositorio de Keras. Para entrenar tu modelo de clasificación, solo corre la célula siguiente.

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# Dimensiones de la imagen
img_rows, img_cols = 28, 28

# Dividir los datos en aprendizaje y evaluación
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convertir los vectores de clase a vectores binarios
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Guarda los pesos (weights) para predicciones privadas futuras

Excelente! Tu modelo ya está entrenado. Vamos a guardar los pesos del modelo con `model.save()`. En el siguiente notebook, vamos a cargar estos pesos en Syft Keras para empezar a proveer predicciones privadas.

In [ ]:
model.save('short-conv-mnist.h5')